In [ ]:
!git clone https://github.com/Anuar20041999/DCUnet-10-with-PyTorch.git DCUnet10
!pip install pypesq
!pip install torchaudio

In [1]:
import librosa
import numpy as np
from IPython.display import Audio
import os
import torchaudio
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

from DCUnet10 import dataset
from DCUnet10 import model10, train

In [ ]:
# if using Colab
from google.colab import drive
drive.mount('/content/drive')

In [2]:
def listdir_fullpath(d):
    return sorted([os.path.join(d, f) for f in os.listdir(d)])

# all files must be with rate==16000, and 10 sec long
clean_path_train = 'path_to_clean_files_train'
noisy_path_train = 'path_to_noisy_files_train'
clean_path_val = 'path_to_clean_files_val(test)'
noisy_path_val = 'path_to_noisy_files_val(test)'
os.makedirs(clean_path_train, exist_ok=True)
os.makedirs(noisy_path_train, exist_ok=True)
os.makedirs(clean_path_val, exist_ok=True)
os.makedirs(noisy_path_val, exist_ok=True)

In [4]:
batch_size = 7
# it returns dictionary of 2 dataloaders - 'train' and 'val'
dataloaders = dataset.make_dataloaders(clean_path_train, noisy_path_train,
                                       clean_path_val, noisy_path_val, batch_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = model10.ComplexUNet().to(device) # initialize model

lr=5e-3
optimizer_ft = optim.Adam(model.parameters(), lr=lr)


step_size, gamma = 10, 0.1
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=step_size, gamma=gamma)

num_epochs = 25
#start train
model = train.train_model(model, optimizer_ft, exp_lr_scheduler, dataloaders=dataloaders, num_epochs=num_epochs, save_best=True)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.eval()

In [8]:
from pypesq import pesq

#eval pesq with pypesq
def eval_pesq(dataloader):
    score = 0
    for i in range(len(dataloader)):
        clean, noised = next(iter(dataloader))
        clean = clean.to(device)
        noised = noised.to(device)
        pred = model(noised)

        clean = clean.cpu().numpy()
        noised = noised.cpu().numpy()
        pred = pred.cpu().detach().numpy()

        for j, k in zip(clean, pred):
            score += pesq(j, k, 16000)
    score /= (batch_size*len(dataloader))
    return score

In [ ]:
print(eval_pesq(dataloaders['val']))